<a href="https://colab.research.google.com/github/xrjiang83/learn_d2l/blob/master/Learn_D2L_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet-cu100

     |████████████████████████████████| 540.1MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [15]:
import mxnet
from mxnet import nd

X = nd.shuffle(nd.arange(10)).reshape((2,5)) # batch_size = 2, num_steps = 5
for x in X.T : # 每一個step生成一個one-hot向量
  one_hot = nd.one_hot(x, 10)
  print(one_hot)

#inputs = to_onehoe()


[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
<NDArray 2x10 @cpu(0)>

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
<NDArray 2x10 @cpu(0)>

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 2x10 @cpu(0)>

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
<NDArray 2x10 @cpu(0)>

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
<NDArray 2x10 @cpu(0)>


In [20]:
# 从Github下载周杰伦歌词文件
!pip install svn

     |████████████████████████████████| 163kB 7.2MB/s 
  Created wheel for svn: filename=svn-0.3.46-py2.py3-none-any.whl size=14756 sha256=82b09ec1e13c5ab7f185b8678aa6e5830fa9d8d9049913d9e96c6c56b13339e0
  Stored in directory: /root/.cache/pip/wheels/34/42/01/edad385df195a2e01f0743cb078335abfa9e6b53e476b3feda
Successfully built svn


In [23]:
!/usr/local/bin/svn --version

/bin/bash: /usr/local/bin/svn: No such file or directory


In [25]:
# 从Github下载周杰伦歌词文件
!git clone https://github.com/xrjiang83/d2lzh_data.git

Cloning into 'd2lzh_data'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [32]:
!ls d2lzh_data


airfoil_self_noise.dat	kaggle_cifar10		    kaggle_house_pred_train.csv
fr-en-small.txt		kaggle_dog		    ptb.zip
jaychou_lyrics.txt.zip	kaggle_house_pred_test.csv


In [33]:
# 读取数据集
from mxnet import nd
import random
import zipfile

with zipfile.ZipFile('d2lzh_data/jaychou_lyrics.txt.zip') as zin :
  with zin.open('jaychou_lyrics.txt') as f :
    corpus_chars = f.read().decode('utf-8')

corpus_chars[0:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [38]:
# 把训练集中的回车换行全部换成空格
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')

# 建立字符索引
idx_to_char = list(set(corpus_chars)) # 词典数组
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print(len(idx_to_char), len(char_to_idx))

# 将训练数据集中每个字符转化为索引，并打印钱20个字符及其对应的索引
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)


2582 2582
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [1694, 378, 623, 1519, 1124, 1321, 814, 1694, 378, 198, 1159, 1226, 2321, 577, 1191, 1736, 814, 1694, 378, 198]


In [40]:
# 给定一个输入小批量的字符索引序列：X - batch_size x num_steps
# 生成该每一个时间步下该小批量的one-hot特征向量：XO - num_steps x (batch_size x vocab_size)
def to_onehot(X, size) : 
  return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2,5))
inputs = to_onehot(X, 50)
len(inputs), inputs[0].shape

(5, (2, 50))

In [44]:
# 随机采样函数，返回
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None) : 
  # 减1是因为输出的索引是相应输入索引加1：考虑到RNN用来预测下一个字，所以最后一个字不作为输入，只作为训练时的预测输出
  num_examples = (len(corpus_indices) - 1) // num_steps  # 每一时间步序列作为一个样本（实例）
  epoch_size = num_examples // batch_size  # 每一个小批量里含若干个样本（实例）
  example_indices = list(range(num_examples))
  random.shuffle(example_indices)

  # 返回从pos开始长度为num_steps的序列
  def _data(pos) : 
    return corpus_indices[pos : pos + num_steps]
  
  for i in range(epoch_size) : # 生成这么多组小批量
    # 每次读取batch_size个随机样本
    i = i * batch_size
    batch_indices = example_indices[i : i + batch_size]
    X = [_data(j * num_steps) for j in batch_indices]
    Y = [_data(j * num_steps + 1) for j in batch_indices]
    yield nd.array(X, ctx), nd.array(Y, ctx)

my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6) : 
  print('X:', X, '\nY:', Y)


X: 
[[ 0.  1.  2.  3.  4.  5.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)>
X: 
[[ 6.  7.  8.  9. 10. 11.]
 [18. 19. 20. 21. 22. 23.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [19. 20. 21. 22. 23. 24.]]
<NDArray 2x6 @cpu(0)>


In [47]:
def try_gpu() : 
  try: 
    ctx = mxnet.gpu()
    _ = nd.zeros((1,), ctx=ctx)
  except mxnet.base.MXNetError: 
    ctx = mxnet.cpu()
  return ctx

ctx = try_gpu()
ctx


gpu(0)

In [53]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = try_gpu()
print('will use', ctx)

def get_params() : 
  def _one(shape) : 
    return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

  # 隐藏层参数
  W_xh = _one((num_inputs, num_hiddens))
  W_hh = _one((num_hiddens, num_hiddens))
  b_h = nd.zeros(num_hiddens, ctx=ctx)
  # 输出层参数
  W_hq = _one((num_hiddens, num_outputs))
  b_q = nd.zeros(num_outputs, ctx=ctx)
  # 附上梯度
  params = [W_xh, W_hh, b_h, W_hq, b_q]
  for param in params :
    param.attach_grad()
  return params

# 定义模型
def init_rnn_state(batch_size, num_hiddens, ctx) : 
  return nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)

# 在一个时间步长内计算隐藏状态和输出
def rnn(inputs, states, params) : 
  # inputs为num_steps个batch_size x vocab_size的（输入）特征矩阵；outputs也是一样的形状
  # states是batch_size x num_hiddens维的隐含状态向量
  W_xh, W_hh, b_h, W_hq, b_q = params
  H = states
  outputs = []
  for X in inputs : # 一个时间步上的输入特征矩阵（矩阵：因为有batch_size个向量堆叠而成）
    H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
    Y = nd.dot(H, W_hq) + b_q
    outputs.append(Y)
  return outputs, H

# 测试rnn函数
states = init_rnn_state(X.shape[0], num_hiddens, ctx)  # X.shape[0]: batch_size
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, states_new = rnn(inputs, states, params)
len(outputs), outputs[0].shape, states_new[0].shape


will use gpu(0)


(6, (2, 2582), (256,))

In [0]:
# 定义预测函数
def predict_simple_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx) : 
  # prefix是输入字符串
  states = init_rnn_state(1, num_hiddens, ctx)
  output = [char_to_idx[prefix[0]]] # 输入字符的索引
  for t in range(num_chars + len(prefix) - 1) : 
    # 将上一时间步的输出作为当前时间步的输入
    X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
    # 计算输出和更新隐藏状态
    Y, states = rnn(X, states, params)
    #
    if t < len(prefix) - 1 : #下一个时间步的输入是prefix中的字符
      output.append(char_to_idx[prefix[t+1]])
    else : # 下一个时间步的输入是当前的最佳预测字符
      output.append(int(Y[0].argmax(axis=1).asscalar()))
  return ''.join([idx_to_char[i] for i in output])

In [60]:
# 测试简单RNN
predict_simple_rnn('分开', 100, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx)


'分开珠灰厅男嘆视醇瑚仓虹迭毅钮右撑弱洒仙約腰叛狱典符倒千压祷苔灭性鍵底楔副左铜窑颓應押四汗跑脂削邦蒙厉供备痹箭谢粥四汗跑脂削邦蒙厉供备痹箭谢粥四汗跑脂削邦蒙厉供备痹箭谢粥四汗跑脂削邦蒙厉供备痹箭谢粥四汗跑'

In [0]:
# 定义模型训练函数
def train_and_predict_simple_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, char_to_idx, is_random_iter, num_epochs, num_steps, lr, )